In [6]:
import pandas as pd
import pickle


In [7]:
# Model was trained on overdose deaths in the state of GA from Jan 2016 to Sept 2020
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [23]:
df = pd.read_csv('prediction_vals.csv')
df.head()

,date,data_value,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,rolling_mean_4,rolling_mean_5,rolling_mean_6,rolling_mean_7,rolling_mean_8,rolling_mean_9,rolling_mean_10,rolling_mean_11,rolling_mean_12,expanding_mean_2
0,1/1/2016,1299.00,1308,1315,1337,1314,1327,1330,1311,1289,...,1314.75,1314.60,1316.67,1318.57,1317.62,1314.44,1310.00,1305.18,1299.33,1292.15
1,2/1/2016,1277.00,1299,1308,1315,1337,1314,1327,1330,1311,...,1299.75,1307.20,1308.33,1311.00,1313.38,1313.11,1310.70,1307.00,1302.83,1291.07
2,3/1/2016,1278.00,1277,1299,1308,1315,1337,1314,1327,1330,...,1290.50,1295.40,1302.33,1304.00,1306.88,1309.44,1309.60,1307.73,1304.58,1290.20
3,4/1/2016,1291.00,1278,1277,1299,1308,1315,1337,1314,1327,...,1286.25,1290.60,1294.67,1300.71,1302.38,1305.11,1307.60,1307.91,1306.33,1290.25
4,5/1/2016,1268.00,1291,1278,1277,1299,1308,1315,1337,1314,...,1278.50,1282.60,1286.83,1290.86,1296.62,1298.56,1301.40,1304.00,1304.58,1288.94


In [44]:
# Convert to datetime
df['date'] = pd.to_datetime(df['date'])

# Get the last three rows for validation. October 2020, November 2020, December 2020
x = df.drop(['data_value'], axis=1).tail(3)

In [60]:
forecast = loaded_model.forecast(x)


In [61]:
# This list contains the forecasts for the last three months of 2020 in order
forecast[0]

array([1778.00618248, 1807.00995213, 1848.01606486])

In [62]:
# Feature columns had to be passed into the model for prediction. This is just dataframe contains all the feature columns - the ones we provided and the ones AzureML automatically added in.
forecast[1]

,,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,...,rolling_mean_12_WASNULL,expanding_mean_2_WASNULL,_automl_year,_automl_half,_automl_quarter,_automl_month,_automl_wday,_automl_qday,_automl_week,_automl_target_col
date,_automl_dummy_grain_col,,,,,,,,,,,,,,,,,,,,,
2020-10-01,_automl_dummy_grain_col,1734.00,1708.00,1702.00,1627.00,1556.00,1507.00,1443.00,1426.00,1387.00,1378.00,...,0,0,2020,2,4,10,3,1,40,1778.01
2020-11-01,_automl_dummy_grain_col,1778.00,1734.00,1708.00,1702.00,1627.00,1556.00,1507.00,1443.00,1426.00,1387.00,...,0,0,2020,2,4,11,6,32,44,1807.01
2020-12-01,_automl_dummy_grain_col,1807.00,1778.00,1734.00,1708.00,1702.00,1627.00,1556.00,1507.00,1443.00,1426.00,...,0,0,2020,2,4,12,1,62,49,1848.02


In [72]:
# Just a final dataframe to compare the results
final = pd.DataFrame({'Date': ['10/01/2020', '11/01/2020', '12/01/2020'],'Actual Overdoses': [1778, 1807, 1848], 'AutoML Predictions': forecast[0], 'CDC Predicions': [1801, 1827, 1863]})
final

,Date,Actual Overdoses,AutoML Predictions,CDC Predicions
0,10/01/2020,1778,1778.01,1801
1,11/01/2020,1807,1807.01,1827
2,12/01/2020,1848,1848.02,1863


In [73]:
# How the passed in data looks. The data_value values for the last three months of 2020 are NaN.
df.tail(3)

,date,data_value,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,rolling_mean_4,rolling_mean_5,rolling_mean_6,rolling_mean_7,rolling_mean_8,rolling_mean_9,rolling_mean_10,rolling_mean_11,rolling_mean_12,expanding_mean_2
57,2020-10-01,nan,1734,1708,1702,1627,1556,1507,1443,1426,...,1730.50,1709.80,1684.17,1658.86,1631.88,1609.00,1586.80,1567.82,1552.25,1405.51
58,2020-11-01,nan,1778,1734,1708,1702,1627,1556,1507,1443,...,1756.75,1745.80,1726.00,1701.71,1677.38,1651.33,1628.80,1606.82,1587.75,1411.17
59,2020-12-01,nan,1807,1778,1734,1708,1702,1627,1556,1507,...,1791.75,1775.00,1762.83,1743.43,1720.00,1696.33,1671.00,1648.73,1626.92,1417.24
